# Using the `BigInteger` to compile utility-scale quantum algorithms

In a previous tutorial, we learned how Jasp allows us to use Qrisp to compile quantum algorithms.
However, Jasp has a caveat that may not be immediately apparent to those coming from Python: it uses fixed-size integers.
On the other hand, Python's `int` supports arbitrary sizes.
For example, let us create a large integer, parse it to Jasp, and return it to Python:

In [20]:
from qrisp import *

a = 2**64

@jaspify
def return_integer(i):
    return i

try:
    b = return_integer(a)
except OverflowError as e:
    print(e)

An overflow was encountered while parsing an argument to a jitted computation, whose argument path is args[0].


Here, Jasp checks the integer for its size before parsing it to the jitted computation and raises an error.
However, not all internal computations are bounds-checked.

In [21]:
from qrisp import *

a = 2**40

@jaspify
def add_to_integer(i):
    return i*2**40

r_jasp = add_to_integer(a)
r_python = a * 2**40
print("Python:", r_python)
print("Jasp:", r_jasp)

Python: 1208925819614629174706176
Jasp: 0


Here, we perform the same computation using Python and Jasp and get different results.
Additionally, we are not alerted to the problem since Jasp does not check bound violations on all internal computations.

This issue is known as _Integer overflow_ and is a common pitfall in many low-level programming languages.
It is a direct consequence of setting the size of the integer to static value, for example, 32- or 64-bit, and many other basic data types are handled the same way.
Doing this allows for well-optimized operations and avoids additional overhead introduced by managing a dynamically-sized data container.
In fact, we are not able to circumvent this restriction in general because of the limitations of Jasp.
Still, to allow us to work with integers that are larger than the native 32- or 64-bit types, we provide the `BigInteger` class.

The idea of the `BigInteger` class is easily explained: represent very large integers that do not fit into the default sizes.
Internally, the `BigInteger` uses an array of 32-bit integers to represent larger values.
As required by Jasp, this array possesses a static size.
Now, we will see how to work with this integer.

## Working classically with `BigInteger`

To create an instance of the `BigInteger` class, we can make use of the `create_static` or `create_dynamic` function.
Where `create_static` is a Python function that is *not jasp-compatible*, the function `create_dynamic` is written in terms of jasp primitives.
This distinction showcases a key problem when working at the intersection of Python and Jasp.
We cannot exactly create a BigInteger that is too large for the native Jasp types when using `create_dynamic` as the used Jasp function require a size-limited Jasp integer.
In these cases, we must use `create_static`.
For example, let us create a 2048-bit integer.

In [22]:
@jaspify
def create_bi():
    return BigInteger.create_dynamic(1, 64) # Works because the value is one

bi_one = create_bi()
bi_large = BigInteger.create_static(2**1200, 64) # Works because we use the static function

For both functions, the first argument is the value of the integer and the second is the number of 32-bit digits (limbs) to allocate for the instance.
Here, we choose 64 since $64\cdot 32 = 2048$.
To obtain the value stored in the integer, we use the `__call__` method, which can be called by opening and closing brackets `()`.

In [23]:
print("bi_large:", bi_large())
print("Correct value:", bi_large() == 2**1200)

bi_large: 17218479456385750618067377696052635483579924745448689921733236816400740691241745619397484537236046173286370919031961587788584927290816661024991609882728717344659503471655990880884679896520055123906467064419056526231345685268240569209892573766037966584735183775739433978714578587782701380797240772477647874555986712746271362892227516205318914435913511141036261376
Correct value: True


In Python-mode, that is, outside any jasp function, the method returns a Python integer of the value stored inside the `BigInteger` instance.
However, when using this method in jasp-mode this is not possible, and thus the method returns an approximation of the value as a 64-bit float.

The `BigInteger` class supports a wide range of operators to perform basic arithmetic operations, comparisons, and bit operations

In [24]:
@jaspify
def add_bi(a, b): return a + b
@jaspify
def sub_bi(a, b): return a - b
@jaspify
def mul_bi(a, b): return a * b
@jaspify
def div_bi(a, b): return a // b
@jaspify
def mod_bi(a, b): return a % b
@jaspify
def pow_bi(a, b): return a ** b
@jaspify
def ge_bi(a, b): return a >= b
@jaspify
def lt_bi(a, b): return a < b
@jaspify
def eq_bi(a, b): return a == b
@jaspify
def lshift_bi(a, b): return a << b
@jaspify
def xor_bi(a, b): return a ^ b

x = BigInteger.create_static(2**1200, 64)
y = BigInteger.create_static(3**600, 64)

print("x+y :", add_bi(x, y)())
print("x-y :", sub_bi(x, y)())
print("x*y :", mul_bi(x, y)())
print("x//y :", div_bi(x, y)())
print("x%y :", mod_bi(x, y)())
print("y**2 :", pow_bi(y, 2)())
print("x>=y :", ge_bi(x, y))
print("x<y :", lt_bi(x, y))
print("x==y :", eq_bi(x, y))
print("x<<2 :", lshift_bi(x, 2)())
print("x^y :", xor_bi(x, y)())

x+y :

 17218479456385750618067377696052635483579924745448689921733236816400740691260484896436332477122800193206729042456270056819577708848783571008203520845886481023385623626125021737692410484491915034285536152112175577316485251485611204293277517379905996129992080893738042135558278052875995146630382082004344231698587579681960846663105331219780109273605735020941393377
x-y : 17218479456385750618067377696052635483579924745448689921733236816400740691223006342358636597349292153366012795607653118757592145732849751041779698919570953665933383317186960024076949308548195213527397976725937475146206119050869934126507630152169937039478286657740825821870879122689407614964099462950951517413385845810581879121349701190857719598221287261131129375
x*y : 836304334102744412146893861148063092043894271261591697109904825098509123215534022872720179170550229988838388187568138701227043327736925801850820248132021386594287640328369471854843032519956377432463466498532201509744936873601263270745479387283680497950296605823

Here, we use functions with the `@jaspify` annotation to showcase that the `BigInteger` works in jasp mode.
For a complete list, see the documentation of the `BigInteger`.

## Finding the order of a `BigInteger`

As we learned in a previous tutorial, the problem of factoring can be reduced to order-finding.
Shor's algorithm does exactly that and provides an efficient quantum algorithm for order-finding.
Following the previous tutorials, we implement this

In [25]:
def find_order(a: BigInteger, N: BigInteger):
    qg = QuantumModulus(N, inpl_adder=gidney_adder)
    qg[:] = 1
    qpe_res = QuantumFloat(2*qg.size + 1)
    h(qpe_res)
    for i in jrange(qpe_res.size):
        with control(qpe_res[i]):
            qg *= a
        a = (a*a)%N
    QFT(qpe_res, inv = True)
    return measure_to_big_integer(qpe_res, a.digits.size)

The function `find_order` provides a jasp-compatible implementation of order-finding that makes use of the BigIntger class.
Note that we use the function `measure_to_big_integer` instead of `measure` to return the measured value of the quantum variable as a big integer.
This use case shows the importance of the `BigInteger` class, since a native jasp integer is limited to 64 bits.
Only minimal modifications are required to make the implementation compatible with the new integer type.

Of course, we cannot run this function for, say, 2048-bit integers on a classical computer.
Still, we are able to perform resource estimation using this implementation.

In [26]:
@count_ops(meas_behavior="0")
def find_order_count_ops(a: BigInteger, N: BigInteger):
    qg = QuantumModulus(N, inpl_adder=gidney_adder)
    qg[:] = 1
    qpe_res = QuantumFloat(2*qg.size + 1)
    h(qpe_res)
    for i in jrange(qpe_res.size):
        with control(qpe_res[i]):
            qg *= a
        a = (a*a)%N
    QFT(qpe_res, inv = True)
    return measure_to_big_integer(qpe_res, a.digits.size)

N = BigInteger.create_static(151557408999110657826917604970069258585, size=10)
a = BigInteger.create_static(10409234017673608357083055217615539999, size=10)
    
ops_count = find_order_count_ops(a, N)
print(ops_count)

{'t_dg': 73603532, 's': 36607456, 't': 73506377, 'cx': 386608045, 'p': 97155, 'x': 1021, 'h': 110017188, 'measure': 36607711}


Using the `count_ops` annotation, we can obtain a dictionary containing the number and type of quantum gates that are used in the implementation.